# 1. Preprocessing of EXTOD 101 for ML 

The goal of this notebook is to prepare the EXTOD 101 data for machine learning - predicting euglycemia during and around exercise.

EXTOD (Exercise in Type One Diabetes) 101 was a trial looking into the glucose control of indivuals with T1D training for the Swansea half marathon. There was around 35 people with 8 weeks worth of FreeStyle Libre data and exercise diaries recording their exercise bouts. There is also demographic data (including age, BMI, years since diagnosis) and lab data (including HbA1c, Cpeptide).

### Objectives
1. Clean and combine the exercise diaries
2. Preprocess demographic data
3. Preprcoess lab data
4. Prepare FGM data

## 1.0. Import packages

In [2]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
from functools import reduce
import os
import preprocess_helper

In [3]:
# Set directories for data
directory_cgm = '../../data/raw_data/extod_101/cgm_data/'
directory_diaries = '../../data/raw_data/extod_101/diary_data/'
directory_demo_lab = '../../data/raw_data/extod_101/'

## 1.1. Clean and combine exercise diaries

### 1.1.2. Combine dataframes

In [4]:
dropped_date = 0

In [5]:
# Empty dataframe
ex_df = pd.DataFrame()
# Loop through all exercise diaries
for filename in os.listdir(directory_diaries):
    filepath = directory_diaries + filename
    df = pd.read_excel(filepath, sheet_name=0).append(pd.read_excel(filepath, sheet_name=1))
    # Split the two side-by-side dfs and join them
    df = preprocess_helper.clean_diaries(df)
    # Set id as first 4 characters of 
    df['ID'] = filename[:4]
    # Fix any dodgy times
    df['What time']= df['What time'].apply(lambda x: preprocess_helper.check_time(x))
    size1= df[df['Exercise']=='yes'].shape[0]
    # Drop any instances that don't have datetime data
    df.dropna(subset=['What time', 'Date', 'how long'], inplace=True)
    dropped_date += (size1-df.shape[0])
    # Combine date and time
    df['start_datetime'] = df.apply(lambda row: pd.Timestamp.combine(row['Date'], row['What time']), axis=1)
    # Convert duration to a float
    #df['how long'] = df['how long'].apply(lambda x: float(x))
    # Add duration to start time to get finish time
    df['finish_datetime'] = df.apply(lambda x: x['start_datetime']
                                     + datetime.timedelta(minutes=x['how long']), axis=1)
    # Drop useless columns and reset index
    df = df.drop(columns=['Date', 'What time', 'how long', 'Exercise'])
    df.reset_index(drop=True,inplace=True)
    # Append to global dataframe
    ex_df = ex_df.append(df)

In [6]:
# Select useful columns and rename them
cols_to_choose = ['ID', 'start_datetime', 'finish_datetime', 'Intensity',
                  'Type of exericse', 'Glucose before', 'Glucose After']
diaries = ex_df.loc[:,cols_to_choose]
diaries.columns = ['ID', 'start_datetime', 'finish_datetime', 'intensity',
                  'type_of_exercise', 'starting_glucose', 'finishing_glucose']

In [7]:
# Set start and finish to datetime
diaries.start_datetime = pd.to_datetime(diaries.start_datetime)
diaries.finish_datetime = pd.to_datetime(diaries.finish_datetime)

In [8]:
dropped_date

-17

### 1.1.3. Sort out dates

In [9]:
# Get breakdown for start datetime
diaries = preprocess_helper.date_preprocessing(diaries, 'start_datetime', False, True, True)
# Calculate duration
diaries['duration'] = (diaries['finish_datetime'] - diaries['start_datetime']).apply(lambda x: x.total_seconds()/60)

In [10]:
diaries

,ID,start_datetime,finish_datetime,intensity,type_of_exercise,starting_glucose,finishing_glucose,month,day,day_of_week,time_of_day,duration
0,3039,2018-05-13 09:00:00,2018-05-13 16:00:00,17.0,walk,11.0,13.0,5,13,6,morning,420.0
1,3039,2018-05-19 09:00:00,2018-05-19 09:28:00,12.0,run,12.0,7.0,5,19,5,morning,28.0
2,3039,2018-05-22 18:00:00,2018-05-22 19:17:00,13.0,run,13.0,7.0,5,22,1,evening,77.0
3,3039,2018-05-28 09:30:00,2018-05-28 10:33:00,17.0,run,11.0,13.0,5,28,0,morning,63.0
4,3039,2018-05-31 17:00:00,2018-05-31 18:30:00,15.0,weights,9.0,5.0,5,31,3,afternoon,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
23,3005,2018-06-30 07:30:00,2018-06-30 11:00:00,11.0,cycle-run-cycle,8.4,4.9,6,30,5,morning,210.0
24,3005,2018-07-02 20:00:00,2018-07-02 20:55:00,10.0,run,10.2,4.5,7,2,0,evening,55.0
25,3005,2018-07-03 19:00:00,2018-07-03 20:00:00,16.0,run,7.0,7.5,7,3,1,evening,60.0
26,3005,2018-07-04 19:00:00,2018-07-04 20:00:00,12.0,run,5.2,3.9,7,4,2,evening,60.0


In [11]:
diaries.shape

(779, 12)

In [12]:
def handle_overlaps(group):
    # Sort by start_datetime
    group = group.sort_values(by='start_datetime')
    
    non_overlapping = []
    overlap_count = 0  # Counter to limit the number of printed overlaps
    for _, row in group.iterrows():
        if non_overlapping:
            last_interval = non_overlapping[-1]
            # Check for overlap
            if row['start_datetime'] < last_interval['finish_datetime']:
                # Print the overlapping intervals (limited to a few for brevity)
                if overlap_count < 5:
                    print(f"Overlap detected for ID {group['ID'].iloc[0]}:")
                    print(f"Interval 1: {last_interval['start_datetime']} to {last_interval['finish_datetime']}")
                    print(f"Interval 2: {row['start_datetime']} to {row['finish_datetime']}\n")
                    overlap_count += 1

                curr_duration = (row['finish_datetime'] - row['start_datetime']).seconds / 60
                last_duration = (last_interval['finish_datetime'] - last_interval['start_datetime']).seconds / 60
                
                # Decide which interval to keep
                if (curr_duration >= 10 and curr_duration <= 120) and (last_duration < 10 or last_duration > 120):
                    non_overlapping[-1] = row
                elif (last_duration >= 10 and last_duration <= 120) and (curr_duration < 10 or curr_duration > 120):
                    continue
                elif curr_duration > last_duration:
                    non_overlapping[-1] = row
            else:
                non_overlapping.append(row)
        else:
            non_overlapping.append(row)
    
    return pd.DataFrame(non_overlapping)

# Apply the function to each group
diaries = diaries.groupby('ID').apply(handle_overlaps).reset_index(drop=True)


In [13]:
len(diaries.ID.unique())

37

### 1.1.4. Categorise exercise types

In [13]:
# Rename all the dodgy words
diaries['type_of_exercise'].replace({'Cycle': 'cycle', 'walking': 'walk', 
                                        'hike':'walk', 'pliates': 'pilates',
                                       'rin': 'run', 'Run':'run', 'ron':'run',
                                       'run ': 'run'}, inplace=True)

In [14]:
size1 = diaries.shape[0]

In [15]:
diaries['type_of_exercise'].value_counts()

run                  464
walk                  78
gym                   58
cycle                 52
exercise class        27
yoga                  24
golf                  16
pilates               13
HIT                    5
rock climbing          5
cycle-run-cycle        5
badminton              3
cross trainer          3
circuits               2
core                   2
painting               2
boxing                 2
interval training      2
run/gym                2
step class             1
dance                  1
weights                1
swim                   1
housework              1
gardening              1
boot camp              1
surfing                1
Trampolining           1
Barre                  1
tough mudder           1
paddle boarding        1
2.6                    1
manual work            1
Name: type_of_exercise, dtype: int64

In [16]:
# Drop the non-exercise activities
#diaries = diaries.loc[~diaries['type_of_exercise'].isin(['housework', 'gardening', 'manual work'])]

In [17]:
# Divide the exercise types into aerobic, anaerobic and mixed
dict_exercise_type = {'run': 'aer', 'walk': 'aer', 'gym': 'mix', 'cycle': 'aer',
                      'swim': 'aer', 'exercise class': 'mix', 'yoga': 'ana', 'pilates': 'ana',
                     'golf': 'aer', 'cycle-run-cycle': 'aer', 'HIT': 'ana', 
                     'rock climbing': 'ana', 'core': 'ana', 'badminton': 'aer',
                     'cross trainer': 'aer', 'painting': 'mix', 'boxing': 'mix',
                     'circuits': 'mix', 'Barre': 'ana', 'Trampolining': 'aer', 
                     '2.6': np.nan,
                     2.6: np.nan, 
                     'interval training': 'mix', 'dance': 'aer',
                     'boot camp': 'mix', 'surfing': 'mix', 'paddle boarding': 'mix',
                      'step class': 'aer','tough mudder': 'mix', 'run/gym':'mix', 'weights':'ana',
                      'housework':'mix', 'gardening':'mix', 'manual work':'mix'
                  }
diaries['form_of_exercise'] = diaries['type_of_exercise'].replace(dict_exercise_type)

In [18]:
# Change intensity to range 0-2
diaries['intensity'] = diaries['intensity'].apply(lambda x: preprocess_helper.replace_borg(x))

In [19]:
size1

779

In [20]:
size1 - diaries.shape[0]

0

## 1.2. Preprocess demographics data

In [21]:
# Upload demographic data
df_demo = pd.read_excel(directory_demo_lab+'demographics.xlsx', sheet_name='Demographics')
# Remove ones that don't have data
df_demo = df_demo.loc[df_demo['Questionnaire 1'] == 'Yes']

### 1.2.1. Calculate BMI

In [22]:
# Fill null values in order to manipulate columns
df_demo.loc[:,['Metres', 'Centimetres', 'Feet ', 'Inches']].fillna(0, inplace=True)

In [23]:
# Height recorded in different metrics so need to be sorted out (bloody medics)
df_demo['height_cm'] = df_demo['Metres']*100 + df_demo['Centimetres']
df_demo['height_inch'] = df_demo['Feet ']*12 + df_demo['Inches']
df_demo['height_inch'] = df_demo['height_inch']*2.54
df_demo['height_inch'].replace(0, np.nan, inplace=True)
df_demo['height'] = df_demo['height_inch'].fillna(df_demo['height_cm'])

In [24]:
# Ditto for weight
df_demo['weight_pounds'] = (df_demo.Stone*14 + df_demo.Pounds)*0.454
df_demo.weight_pounds.replace(0, np.nan, inplace=True)
df_demo['weight'] = df_demo.weight_pounds.fillna(df_demo.Kilograms)

In [25]:
# Calculate bmi
df_demo['bmi'] = df_demo['weight']/(df_demo['height']/100*df_demo['height']/100)

In [26]:
# Drop unneeded columns
df_demo.drop(columns = ['Stone', 'Pounds', 'Kilograms', 'Feet ', 'Inches',
                        'Metres', 'Centimetres', 'height_cm', 'height_inch',
                        'Questionnaire 1', 'date entered study'], inplace=True)

### 1.2.2. Change col names and drop redundant cols

In [27]:
# Rename cols
column_names = {'Participant No.': 'ID', 'Age (years)': 'age', 
                'Sex (1=m, 2=F)': 'sex', 'Diagnosed Month': 'diagnosis_month', 
                'Diagnosed Year': 'diagnosis_year', 'length of diabetes years': 
                'years_since_diagnosis', 'date diagnosed with Diabetes': 
                'diagnosis_date','Treat Diabetes 1= MDI, 2 Pump': 
                'insulin_modality', 'Insulin in Pump': 'insulin_in_pump', 
                'fast acting insulin': 'fast_insulin', 'Lon acting Insulin':
                'long_insulin', 'Once or Twice': 'once_or_twice',
                'Total BI': 'total_bi', 'Ratios (1= yes, 2 = no)': 'ratios',
                'Correction Factor': 'correction_factor', 
                'Eyes_1 1= yes, 2 No': 'eyes_1',
               }
df_demo.rename(column_names, axis=1, inplace=True)

In [28]:
# Drop cols
columns_drop = ['DOB', 'diagnosis_date', 'insulin_in_pump', 'fast_insulin', 
                'long_insulin']
df_demo.drop(columns=columns_drop, inplace=True)

In [29]:
# Reset index
df_demo.reset_index(inplace=True, drop=True)

In [30]:
# Replace binary with categorical
df_demo.sex.replace({1:'male', 2:'female'}, inplace=True)
df_demo.insulin_modality.replace({1:'mdi', 2:'pump'}, inplace=True)

In [31]:
df_demo_cut = df_demo.loc[:,['ID', 'age', 'sex', 'years_since_diagnosis', 'insulin_modality', 'bmi']]

## 1.3. Preprocess lab data

In [32]:
# Upload the lab data
df_lab = pd.read_excel(directory_demo_lab + 'demographics.xlsx',
                       sheet_name='Lab results')

In [33]:
# Drop unnecessary columns
to_drop = ['Date of Receipt','Urine C-peptide/Creatinine Ratio (nmol/mmol)', 
           'Date of Receipt', 'Date of Receipt2', 'Date of Receipt3',
           'HBAIC Sample ID', 'Barcode Set', 'Urine Sample ID',
           'Plasma Sample ID']
df_lab.drop(columns=to_drop, inplace=True)
# Rename columns
df_lab.columns = ['ID', 'visit', 'urine_cpep', 'urine_creat', 'hba1c', 'chol',
                 'cpep', 'h_index', 'hdl', 'ldl', 'nhdl', 'trig']

In [34]:
# Replace missing first visits with second visit results
df_lab = df_lab.sort_values(['ID', 'visit']).reset_index(drop=True).groupby('ID').apply(preprocess_helper.get_lab_results)
# Select only first visit results
df_lab = df_lab[df_lab['visit']==1]

In [35]:
# Merge demographic and lab data
demo_lab = df_demo_cut.merge(df_lab, how='outer', on='ID').drop(columns='visit')
demo_lab['ID'] = demo_lab['ID'].astype(int)

In [36]:
demo_lab.head()

,ID,age,sex,years_since_diagnosis,insulin_modality,bmi,urine_cpep,urine_creat,hba1c,chol,cpep,h_index,hdl,ldl,nhdl,trig
0,3001,41.615332,female,10.064339,pump,23.112472,0.03,2.9,73.0,5.2,3.0,339.0,1.62,2.85,3.53,1.48
1,3002,48.008214,female,17.065024,mdi,23.353250,0.03,5.6,57.0,4.3,3.0,59.0,1.88,1.94,2.39,0.99
2,3003,52.049281,male,48.980151,pump,23.205444,0.03,2.4,52.0,4.3,3.0,132.0,1.97,2.15,2.37,0.49
3,3004,30.047912,male,21.560575,mdi,24.957697,0.03,3.3,55.0,3.5,3.0,33.0,1.31,1.87,2.22,0.77
4,3005,47.775496,male,4.065708,mdi,26.523164,2.60,3.4,56.0,4.8,366.0,242.0,1.00,2.39,3.77,3.02


## 1.4. Preprocess FGM data

In [37]:
# Declare empty dataframe
cgm_total= pd.DataFrame()
#loop through every filename in the directory
for filename in os.listdir(directory_cgm):
    print(filename)
    # set filepath for each file in directory
    filepath = directory_cgm+'/'+filename
    # Get the ID from the filename by selecting last 4 characters
    ID = int(filename[:4])
    # Upload cgm dataset for the file
    df = pd.read_csv(filepath, header=2)
    # Set the timestamp to datetime and sort
    df['Meter Timestamp'] = pd.to_datetime(df['Meter Timestamp'], dayfirst=True)
    df = df.sort_values('Meter Timestamp', ascending=True)
    # Set a column 'ID' in the dataframe as the ID so it can be identified later on
    df['ID'] = ID
    cgm_total = cgm_total.append(df)

3016 2.csv
3034 2.csv
3026 2.csv
3031 2.csv
3032 2.csv
3046 2.csv
3020 2.csv
3023 2.csv
3001 2.csv
3045 2.csv
3039 2.csv
3010 2.csv
3019 2.csv
3047 2.csv
3041 2.csv
3044 2.csv
3011 2.csv
3015 2.csv
3004 2.csv
3005 2.csv
3013 2.csv
3038 2.csv
3036 2.csv
3027 2.csv
3003 2.csv
3007 2.csv
3021 2.csv
3024 2.csv
3040 2.csv
3012 2.csv
3014 2.csv
3008 2.csv
3002 2.csv
3037 2.csv
3033 2.csv
3018 2.csv
3043 2.csv
3017 2.csv
3009 2.csv
3030 2.csv
3028 2.csv
3035 2.csv


In [38]:
cgm_data = cgm_total.loc[:,['ID', 'Meter Timestamp','Historic Glucose(mmol/L)','Scan Glucose(mmol/L)',]]
cgm_data.columns = ['ID', 'time', 'glc', 'scan_glc']
cgm_data.dropna(how='all', subset=['glc', 'scan_glc'], inplace=True)
cgm_data.reset_index(inplace=True, drop=True)
cgm_data = cgm_data[~cgm_data[['time', 'ID']].duplicated(keep='first')]

In [39]:
cgm_data.head()

,ID,time,glc,scan_glc
0,3016,2018-06-05 14:46:00,10.3,NaN
1,3016,2018-06-05 14:55:00,NaN,9.8
2,3016,2018-06-05 15:01:00,9.8,NaN
3,3016,2018-06-05 15:03:00,NaN,10.1
4,3016,2018-06-05 15:07:00,NaN,9.9


## 1.5. Save files

In [40]:
# Add 'helm' to IDs to make them unique
for i in [diaries, cgm_data, demo_lab]:
    i['ID'] = i['ID'].apply(lambda row: 'ext_101_'+str(row))

In [41]:
df_directory = '../../data/tidy_data/extod_101/'
diaries.to_csv(df_directory+'exercise.csv', index=False)
cgm_data.to_csv(df_directory+'cgm.csv', index=False)
demo_lab.to_csv(df_directory+'demo_lab.csv', index=False)

In [42]:
# Convert arrays to sets
set1 = set(cgm_data.ID.unique())
set2 = set(diaries.ID.unique())

# Find items that are not common in both arrays
unique_to_array1 = set1 - set2
unique_to_array2 = set2 - set1

In [43]:
# Only have
unique_to_array1

{'ext_101_3011',
 'ext_101_3023',
 'ext_101_3024',
 'ext_101_3035',
 'ext_101_3041',
 'ext_101_3043',
 'ext_101_3044'}

In [44]:
unique_to_array2 = list(unique_to_array2)

In [45]:
size1 = diaries.shape[0]

In [46]:
diaries = diaries[(diaries['duration']>10)&(diaries['duration']<120)]
size1 - diaries.shape[0]

105

In [47]:
filtered_diaries = diaries[~diaries['ID'].isin(unique_to_array2)]


In [48]:
len(filtered_diaries.ID.unique())

35

In [49]:
diaries.shape[0] -filtered_diaries.shape[0]

47